### Se va a llevar a cabo el desarrollo del entrenamiento de 3 tipos de algoritmos basados en árboles
1. Random Forest
2. XGBoost
3. CatBoost
### Para ello, a demás de entrenarlo se va a presentar la matriz de confusión y las métricas obtenidas

## 1. Carga y Exploración del Dataset

- **Carga del CSV:**  
  Se leyó el archivo CSV para tener una primera visualización de los datos.

- **Exploración inicial:**  
  - Se mostraron las primeras filas del dataset.
  - Se revisó la información general (tipos de datos, cantidad de valores nulos) y se obtuvieron estadísticas descriptivas.

---

## 2. Análisis de las Variables

- **Identificación de columnas no informativas:**  
  - `id` y `LoanNr_ChkDgt` son identificadores y no aportan valor predictivo.
  - `Name` contiene información variable sin relevancia directa, salvo que se extraiga información adicional (como sector o tipo de empresa).

- **Variables de fecha:**  
  - `ApprovalDate` y `DisbursementDate` podrían generar fuga de información si se usan tal cual.  
  - Se recomienda transformarlas (por ejemplo, extrayendo el año, mes, etc.) o descartarlas.

---

## 3. Selección de Variables Relevantes

- **Variables potencialmente importantes para predecir la aprobación de un crédito:**  
  - **Características de la empresa:**  
    - `State`, `ApprovalFY`  
    - `NoEmp` (número de empleados)  
    - `NewExist` (empresa nueva o existente)  
    - `CreateJob` y `RetainedJob` (impacto en el empleo)  
    - `FranchiseCode`  
    - `UrbanRural`  
    - `RevLineCr`, `LowDoc`  
    - `DisbursementGross`, `BalanceGross`
  
  - **Variables relacionadas con el banco y la ubicación:**  
    - `Bank`, `BankState`, `City`  
    - Pueden ser relevantes pero es necesario codificarlas o agruparlas correctamente.

---

## 4. Análisis de Correlaciones

- **Matriz de correlación:**  
  Se calculó la correlación entre las variables numéricas y la variable objetivo `Accept` para identificar cuáles tienen mayor influencia en la aprobación del crédito.
  
- **Visualización:**  
  Se utilizó un mapa de calor para visualizar la matriz de correlación y facilitar la identificación de relaciones relevantes.

---

## 5. Preprocesamiento de Datos

- **Eliminación o transformación de columnas:**  
  - Se descartaron columnas como `id`, `LoanNr_ChkDgt` y `Name` en un primer análisis.
  - Las columnas de fechas se pueden transformar o eliminar para evitar data leakage.

- **Codificación de variables categóricas:**  
  - Se aplicó one-hot encoding a variables como `State`, `Bank`, `City` y `BankState` para convertirlas en variables numéricas.

- **Tratamiento de valores nulos:**  
  - Se identificaron y gestionaron los valores nulos, por ejemplo, mediante imputación simple (o descartándolos según el caso).

---

## 6. División del Dataset

- **Separación en conjuntos de entrenamiento y validación:**  
  - Se dividió el dataset en un conjunto de entrenamiento y uno de validación, asegurando mantener la proporción de la variable objetivo (`Accept`) para una evaluación representativa del modelo.

---


## 1. Random Forest

In [37]:
import pandas as pd

# Cargar el dataset de entrenamiento
df = pd.read_csv('../../../data/processed/df_train.csv')

# Mostrar las primeras filas
# print(df.head())
# Información general (tipos de datos y valores nulos)
#print(df.info())

# Estadísticas descriptivas
# print(df.describe(include='all'))


In [38]:
approval_by_city = df.groupby('City')['Accept'].mean().sort_values(ascending=False) 
print("\nTasa de aprobación por ciudad:") 
print(approval_by_city.head(10))


Tasa de aprobación por ciudad:
City
AMSTERDAM                1.0
powell                   1.0
116 EAST THIRD STREET    1.0
ABERDEEN                 1.0
AMLIN                    1.0
Oak Harbor               1.0
Yorkshire                1.0
ZANESVIILLE              1.0
holland                  1.0
Woodmere                 1.0
Name: Accept, dtype: float64


Preprocesamos los datos

In [39]:
cols_to_drop = ['id', 'LoanNr_ChkDgt', 'Name', 'ApprovalDate', 'DisbursementDate', 'State']
df_clean = df.drop(columns=cols_to_drop)
# City, State, Name, Bank, BankState, ApprovalDate, , ApprovalDate, ApprovalFY, NoEmp, NewExist, UrbanRural,DisbursementDate, DisbursementGross

#Codif variables categóricas
df_clean = pd.get_dummies(df_clean, columns=['Bank', 'City', 'BankState'], drop_first=True)

print(df_clean.isnull().sum())
# Ejemplo simple: imputar nulos con 0 (ajusta según cada variable)
df_clean.fillna(0, inplace=True)

# Mostrar la distribución original de la variable target 'Accept'
print("Distribución original de 'Accept':")
print(df_clean['Accept'].value_counts())

# Balancear el DataFrame:
# Supongamos que la clase 0 (rechazados) es la minoritaria y la clase 1 (aceptados) es la mayoritaria.

# Extraer los DataFrames de cada clase
df_accept_0 = df_clean[df_clean['Accept'] == 0]
df_accept_1 = df_clean[df_clean['Accept'] == 1]

# Realizar un muestreo aleatorio de la clase mayoritaria (1) para igualar el número de la minoritaria (0)
n_minority = len(df_accept_0)
df_accept_1_under = df_accept_1.sample(n=n_minority, random_state=42)

# Combinar ambas clases y mezclar los registros
df_balanced = pd.concat([df_accept_0, df_accept_1_under]).sample(frac=1, random_state=42).reset_index(drop=True)

# Verificar la nueva distribución
print("Distribución balanceada de 'Accept':")
print(df_balanced['Accept'].value_counts())

# df_balanced es el DataFrame final balanceado
df_clean = df_balanced

ApprovalFY       0
NoEmp            0
NewExist        14
CreateJob        0
RetainedJob      0
                ..
BankState_VA     0
BankState_WA     0
BankState_WI     0
BankState_WV     0
BankState_WY     0
Length: 2016, dtype: int64
Distribución original de 'Accept':
Accept
1    19004
0     3831
Name: count, dtype: int64
Distribución balanceada de 'Accept':
Accept
1    3831
0    3831
Name: count, dtype: int64


División del dataset

In [ ]:
from sklearn.model_selection import train_test_split

X = df_clean.drop('Accept', axis=1)
y = df_clean['Accept']


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)


Entrenamiento Random Forest

In [45]:
from sklearn.ensemble import RandomForestClassifier

class_weights = {0: 15, 1: 1}  # Ajusta estos valores según tus datos
rf = RandomForestClassifier(n_estimators=100, random_state=42, class_weight=class_weights)
rf.fit(X_train, y_train)

RandomForestClassifier(class_weight={0: 15, 1: 1}, random_state=42)

Evaluación

In [46]:
from sklearn.metrics import confusion_matrix, classification_report, f1_score

y_pred = rf.predict(X_test)

# Matriz de Confusión
cm = confusion_matrix(y_test, y_pred)
print("Matriz de Confusión:")
print(cm)

print(f"Clase 0 SON CRÉDITOS RECHAZADOS y Clase 1 son CRÉDITOS ACEPTADOS")

# Reporte de Clasificación (precision, recall, f1-score, etc.)
print("Reporte de Clasificación:")
print(classification_report(y_test, y_pred))

# Calcular el Macro F1-Score
macro_f1 = f1_score(y_test, y_pred, average='macro')
print(f"Macro F1-Score: {macro_f1:.2f}")

Matriz de Confusión:
[[281 103]
 [127 256]]
Clase 0 SON CRÉDITOS RECHAZADOS y Clase 1 son CRÉDITOS ACEPTADOS
Reporte de Clasificación:
              precision    recall  f1-score   support

           0       0.69      0.73      0.71       384
           1       0.71      0.67      0.69       383

    accuracy                           0.70       767
   macro avg       0.70      0.70      0.70       767
weighted avg       0.70      0.70      0.70       767

Macro F1-Score: 0.70


### Y esto es para subirlo al kaggle

In [ ]:
import pandas as pd
import datetime

# Cargar el dataset de test
df_test = pd.read_csv("../../../data/processed/df_test_nolabel.csv")

# --- Preprocesamiento ---
# Debes replicar exactamente los pasos que aplicaste al entrenamiento.
# 1. Eliminar las columnas irrelevantes (como hiciste en entrenamiento)
cols_to_drop = ['id', 'LoanNr_ChkDgt', 'Name', 'ApprovalDate', 'DisbursementDate', 'State']
df_test_clean = df_test.drop(columns=cols_to_drop)

# 2. Aplicar one-hot encoding a las columnas categóricas que usaste en entrenamiento:
#    En tu entrenamiento usaste: ['Bank', 'City', 'BankState'] con drop_first=True.
df_test_clean = pd.get_dummies(df_test_clean, columns=['Bank', 'City', 'BankState'], drop_first=True)

# 3. Imputar valores nulos (igual que en entrenamiento)
df_test_clean.fillna(0, inplace=True)

# --- Alinear las features ---
# Durante el entrenamiento, tu DataFrame final balanceado 'df_clean' tenía un cierto conjunto de columnas (features).
# Asumiendo que entrenaste el modelo con:
#   X_train = df_clean.drop('Accept', axis=1)
# Guarda la lista de features de entrenamiento:
features = list(df_clean.drop('Accept', axis=1).columns)
print("Número de features en entrenamiento:", len(features))

# Reindexa el DataFrame de test para que tenga exactamente las mismas columnas, llenando con 0 las que no estén.
df_test_clean = df_test_clean.reindex(columns=features, fill_value=0)
print("Número de features en test:", df_test_clean.shape[1])

# --- Predicción ---
# Extrae el array de features del test
X_test = df_test_clean.values

# Realiza las predicciones usando tu modelo entrenado (en este caso, 'rf')
df_test['Accept'] = rf.predict(X_test)

# Asegúrate de que la columna 'Accept' sea de tipo entero
df_test['Accept'] = df_test['Accept'].astype(int)

# --- Exportar Submission ---
# Crea el archivo CSV con las columnas requeridas: 'id' y 'Accept'
filename = f"random_forest_{datetime.datetime.now().strftime('%Y%m%d')}.csv"
df_test.to_csv(filename, columns=['id', 'Accept'], index=False)

print("Archivo de submission 'my-model.csv' generado correctamente.")


Número de features en entrenamiento: 2015
Número de features en test: 2015


/home/angel/LBBYs/.venv/lib/python3.13/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


Archivo de submission 'my-model.csv' generado correctamente.


## Ahora pruebo el Grid para mejorarlo

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, classification_report, f1_score, make_scorer

# 1. Cargar el dataset y preprocesar
df = pd.read_csv('../../../data/processed/df_train.csv')

# Eliminar columnas irrelevantes
cols_to_drop = ['id', 'LoanNr_ChkDgt', 'Name', 'ApprovalDate', 'DisbursementDate', 'State']
df_clean = df.drop(columns=cols_to_drop)

# Aplicar one-hot encoding a las columnas categóricas: Bank, City y BankState
df_clean = pd.get_dummies(df_clean, columns=['Bank', 'City', 'BankState'], drop_first=True)

# Imputar valores nulos
df_clean.fillna(0, inplace=True)

# Balancear el DataFrame (undersampling de la clase mayoritaria)
df_accept_0 = df_clean[df_clean['Accept'] == 0]
df_accept_1 = df_clean[df_clean['Accept'] == 1]
n_minority = len(df_accept_0)
df_accept_1_under = df_accept_1.sample(n=n_minority, random_state=42)
df_balanced = pd.concat([df_accept_0, df_accept_1_under]).sample(frac=1, random_state=42).reset_index(drop=True)
df_clean = df_balanced

# 2. Separar features y target
X = df_clean.drop('Accept', axis=1)
y = df_clean['Accept']

# 3. Dividir en conjuntos de entrenamiento y prueba (stratificado)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# 4. Definir el grid de hiperparámetros para RandomForestClassifier
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2', None]
}

# Crear un scorer basado en el Macro F1-Score para evaluar el balance entre las clases
scorer = make_scorer(f1_score, average='macro')

# 5. Configurar y ejecutar GridSearchCV
grid_rf = GridSearchCV(
    estimator=RandomForestClassifier(random_state=42, class_weight={0: 15, 1: 1}),
    param_grid=param_grid,
    scoring=scorer,
    cv=5,
    n_jobs=-1
)
grid_rf.fit(X_train, y_train)

print("Mejores parámetros encontrados:")
print(grid_rf.best_params_)

# 6. Evaluar el mejor modelo en el conjunto de prueba
best_rf = grid_rf.best_estimator_
y_pred = best_rf.predict(X_test)

print("Matriz de Confusión:")
print(confusion_matrix(y_test, y_pred))
print("\nReporte de Clasificación:")
print(classification_report(y_test, y_pred))
print("Macro F1-Score:", f1_score(y_test, y_pred, average='macro'))


Mejores parámetros encontrados:
{'max_depth': None, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}
Matriz de Confusión:
[[567 200]
 [261 505]]

Reporte de Clasificación:
              precision    recall  f1-score   support

           0       0.68      0.74      0.71       767
           1       0.72      0.66      0.69       766

    accuracy                           0.70      1533
   macro avg       0.70      0.70      0.70      1533
weighted avg       0.70      0.70      0.70      1533

Macro F1-Score: 0.6987897683319517


### Este es un random

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, classification_report, f1_score, make_scorer
from sklearn.feature_selection import RFE

# 1. Cargar el dataset y preprocesar
df = pd.read_csv('../../../data/processed/df_train.csv')
cols_to_drop = ['id', 'LoanNr_ChkDgt', 'Name', 'ApprovalDate', 'DisbursementDate', 'State']
df_clean = df.drop(columns=cols_to_drop)
df_clean = pd.get_dummies(df_clean, columns=['Bank', 'City', 'BankState'], drop_first=True)
df_clean.fillna(0, inplace=True)

# Balancear el DataFrame (undersampling de la clase mayoritaria)
df_accept_0 = df_clean[df_clean['Accept'] == 0]
df_accept_1 = df_clean[df_clean['Accept'] == 1]
n_minority = len(df_accept_0)
df_accept_1_under = df_accept_1.sample(n=n_minority, random_state=42)
df_balanced = pd.concat([df_accept_0, df_accept_1_under]).sample(frac=1, random_state=42).reset_index(drop=True)
df_clean = df_balanced

# 2. Separar features y target
X = df_clean.drop('Accept', axis=1)
y = df_clean['Accept']

# 3. Dividir en entrenamiento y prueba (stratificado)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# 4. Selección de características con RFE
# Usamos un RandomForestClassifier base para estimar la importancia de features
base_rf = RandomForestClassifier(n_estimators=100, random_state=42, class_weight={0:15, 1:1})
# Seleccionaremos, por ejemplo, las 100 mejores features (ajusta según el número total de columnas)
rfe_selector = RFE(estimator=base_rf, n_features_to_select=100, step=0.1)
rfe_selector.fit(X_train, y_train)

# Reducir X_train y X_test a las features seleccionadas
X_train_rfe = X_train.loc[:, rfe_selector.support_]
X_test_rfe = X_test.loc[:, rfe_selector.support_]

print("Número de features seleccionadas:", X_train_rfe.shape[1])

# 5. Definir el grid de hiperparámetros para el RandomForest con las features seleccionadas
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2', None]
}

scorer = make_scorer(f1_score, average='macro')

grid_rf = GridSearchCV(
    estimator=RandomForestClassifier(random_state=42, class_weight={0:15, 1:1}),
    param_grid=param_grid,
    scoring=scorer,
    cv=5,
    n_jobs=-1
)
grid_rf.fit(X_train_rfe, y_train)

print("Mejores parámetros encontrados:")
print(grid_rf.best_params_)

# 6. Evaluar el mejor modelo en el conjunto de prueba
best_rf = grid_rf.best_estimator_
y_pred = best_rf.predict(X_test_rfe)

print("Matriz de Confusión:")
print(confusion_matrix(y_test, y_pred))
print("\nReporte de Clasificación:")
print(classification_report(y_test, y_pred))
print("Macro F1-Score:", f1_score(y_test, y_pred, average='macro'))


## 2. XGBoost

In [62]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import confusion_matrix, classification_report, f1_score

# 1. Cargar el dataset de entrenamiento
df = pd.read_csv('../../../data/processed/df_train.csv')
cols_to_drop = ['id', 'LoanNr_ChkDgt', 'Name', 'ApprovalDate', 'DisbursementDate', 'State']
df_clean = df.drop(columns=cols_to_drop)

# 2. Codificar variables categóricas
# Aplicamos one-hot encoding a las columnas 'Bank', 'City' y 'BankState'
df_clean = pd.get_dummies(df_clean, columns=['Bank', 'City', 'BankState'], drop_first=True)

# 3. Imputar valores nulos
# Rellenamos los valores nulos con 0 (puedes ajustar según cada variable)
df_clean.fillna(0, inplace=True)

# 4. Separar características (X) y variable objetivo (y)
X = df_clean.drop('Accept', axis=1)
y = df_clean['Accept']

# 5. Dividir el dataset de forma estratificada (80% entrenamiento, 20% prueba)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# 6. Crear pesos de muestra para la clase deseada
# Se asigna un peso de 15 a la clase 0 (créditos rechazados) y 1 a la clase 1 (créditos aceptados)
sample_weights = y_train.map({0: 15, 1: 1})

# 7. Inicializar y entrenar el modelo XGBoost
# use_label_encoder=False y eval_metric='logloss' evitan advertencias de versiones recientes
xgb = XGBClassifier(n_estimators=100, random_state=42, use_label_encoder=False, eval_metric='logloss')
xgb.fit(X_train, y_train, sample_weight=sample_weights)

# 8. Realizar predicciones en el conjunto de prueba
y_pred = xgb.predict(X_test)

# 9. Evaluar el modelo

# Matriz de Confusión
cm = confusion_matrix(y_test, y_pred)
print("Matriz de Confusión:")
print(cm)

print("Clase 0 SON CRÉDITOS RECHAZADOS y Clase 1 son CRÉDITOS ACEPTADOS")

# Reporte de Clasificación
print("Reporte de Clasificación:")
print(classification_report(y_test, y_pred))

# Calcular el Macro F1-Score
macro_f1 = f1_score(y_test, y_pred, average='macro')
print(f"Macro F1-Score: {macro_f1:.2f}")


/home/angel/LBBYs/.venv/lib/python3.13/site-packages/xgboost/training.py:183: UserWarning: [01:26:32] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Matriz de Confusión:
[[ 690   76]
 [1980 1821]]
Clase 0 SON CRÉDITOS RECHAZADOS y Clase 1 son CRÉDITOS ACEPTADOS
Reporte de Clasificación:
              precision    recall  f1-score   support

           0       0.26      0.90      0.40       766
           1       0.96      0.48      0.64      3801

    accuracy                           0.55      4567
   macro avg       0.61      0.69      0.52      4567
weighted avg       0.84      0.55      0.60      4567

Macro F1-Score: 0.52


No me funciona catboost porque estoy usando la versión 3.13.1 y tiene que ser inferior a la versión 3.13

## 3. CatBoost

In [10]:
%pip install catboost

  Using cached catboost-1.2.7.tar.gz (71.5 MB)
  Installing build dependencies ... error
  error: subprocess-exited-with-error
  
  × pip subprocess to install build dependencies did not run successfully.
  │ exit code: 1
  ╰─> [143 lines of output]
        Using cached setuptools-78.1.0-py3-none-any.whl.metadata (6.6 kB)
        Using cached wheel-0.45.1-py3-none-any.whl.metadata (2.3 kB)
        Using cached jupyterlab-3.6.8-py3-none-any.whl.metadata (12 kB)
        Using cached conan-1.66.0.tar.gz (789 kB)
        Installing build dependencies: started
        Installing build dependencies: finished with status 'done'
        Getting requirements to build wheel: started
        Getting requirements to build wheel: finished with status 'done'
        Preparing metadata (pyproject.toml): started
        Preparing metadata (pyproject.toml): finished with status 'done'
        Using cached ipython-9.0.2-py3-none-any.whl.metadata (4.3 kB)
        Using cached packaging-24.2-py3-none-any.